In [1]:
%load_ext autoreloadfrom 
%autoreload 2

In [22]:
from component.scripts.test import reduceGroups
import ee


In [39]:
group1 = ee.Image.random(1).multiply(2).round()
group2 = ee.Image.random(2).multiply(2).round()
group3 = ee.Image.random(3).multiply(2).round()
image = ee.Image(group1.multiply(3).add(group2))
geometries = ee.FeatureCollection(
    [
        ee.Feature(
            ee.Geometry.Polygon(
                [
                    [
                        [5.262516683862475, 46.39655013792998],
                        [5.262516683862475, 45.63360136660453],
                        [6.580876058862475, 45.63360136660453],
                        [6.580876058862475, 46.39655013792998],
                    ]
                ],
                None,
                False,
            ),
            {"system:index": "0"},
        ),
        ee.Feature(
            ee.Geometry.Polygon(
                [
                    [
                        [9.569157308862474, 47.74335902540055],
                        [9.569157308862474, 46.99933410813134],
                        [10.931461996362474, 46.99933410813134],
                        [10.931461996362474, 47.74335902540055],
                    ]
                ],
                None,
                False,
            ),
            {"system:index": "1"},
        ),
        ee.Feature(
            ee.Geometry.Polygon(
                [
                    [
                        [13.084782308862474, 46.6082900232144],
                        [13.084782308862474, 45.87889028991833],
                        [14.315251058862474, 45.87889028991833],
                        [14.315251058862474, 46.6082900232144],
                    ]
                ],
                None,
                False,
            ),
            {"system:index": "2"},
        ),
    ]
)

reducer = (
    # ee.Reducer.mean()
    # .combine(ee.Reducer.sum(), None, True)
    ee.Reducer.sum()
        .group(1, "group0")
        .group(2, "group1")
        .group(3, "group2")
)

reduceRegion = (
    image.addBands(group1)
    .addBands(group2)
    .addBands(group3)
    .reduceRegion(
        **{
            "reducer": reducer,
            "geometry": geometries.geometry(),
            "scale": 1000,
        }
    )
)

reduceRegions = (
    image.addBands(group1)
    .addBands(group2)
    .addBands(group3)
    .reduceRegions(**{"collection": geometries, "reducer": reducer, "scale": 1000})
)

In [40]:
reduceRegions.getInfo()

{'type': 'FeatureCollection',
 'columns': {'groups': 'List<Dictionary<Object>>', 'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Polygon',
    'coordinates': [[[5.262516683862475, 45.63360136660453],
      [6.580876058862475, 45.63360136660453],
      [6.580876058862475, 46.39655013792998],
      [5.262516683862475, 46.39655013792998],
      [5.262516683862475, 45.63360136660453]]]},
   'id': '0',
   'properties': {'groups': [{'group2': 0,
      'groups': [{'group1': 0,
        'groups': [{'group0': 0, 'sum': 0},
         {'group0': 1, 'sum': 1212.3294117647056},
         {'group0': 2, 'sum': 1177.6470588235293}]},
       {'group1': 1,
        'groups': [{'group0': 0, 'sum': 386.69019607843126},
         {'group0': 1, 'sum': 3167.7333333333354},
         {'group0': 2, 'sum': 2752.78431372549}]},
       {'group1': 2,
        'groups': [{'group0': 0, 'sum': 353.42745098039217},
         {'group0': 1, 'sum': 2036.5490196078435},

In [44]:
reduceRegionsSum = reduceGroups(
    **{
        "reducer": ee.Reducer.sum().forEach(["mean_sum", "sum_sum"]),
        "featureCollection": reduceRegions,
        "groupKeys": ["group0", "group1", "group2"],
    }
)

EEException: Collection.reduceColumns: Need 3 inputs for Reducer.group(Reducer.forEach(Reducer.sum)), got 2.

In [37]:
reduceRegion.getInfo()

{'groups': [{'group2': 0,
   'groups': [{'group1': 0,
     'groups': [{'group0': 0, 'sum': 0},
      {'group0': 1, 'sum': 3366.329411764705},
      {'group0': 2, 'sum': 3185.0823529411773}]},
    {'group1': 1,
     'groups': [{'group0': 0, 'sum': 1115.105882352941},
      {'group0': 1, 'sum': 9288.831372549032},
      {'group0': 2, 'sum': 7543.968627450977}]},
    {'group1': 2,
     'groups': [{'group0': 0, 'sum': 1090.0156862745096},
      {'group0': 1, 'sum': 5793.764705882355},
      {'group0': 2, 'sum': 4298.164705882352}]}]},
  {'group2': 1,
   'groups': [{'group1': 0,
     'groups': [{'group0': 0, 'sum': 0},
      {'group0': 1, 'sum': 6913.611764705878},
      {'group0': 2, 'sum': 7177.764705882351}]},
    {'group1': 1,
     'groups': [{'group0': 0, 'sum': 2218.494117647062},
      {'group0': 1, 'sum': 17859.827450980443},
      {'group0': 2, 'sum': 15770.615686274508}]},
    {'group1': 2,
     'groups': [{'group0': 0, 'sum': 2105.8666666666663},
      {'group0': 1, 'sum': 11447.

In [38]:
reduceRegionsSum.getInfo()

{'groups': [{'group2': 0,
   'groups': [{'group1': 0,
     'groups': [{'group0': 1}, {'group0': 1, 'sum': 0}]}]},
  {'group2': 0,
   'groups': [{'group1': 0,
     'groups': [{'group0': 1},
      {'group0': 1, 'sum': 0},
      {'group0': 2},
      {'group0': 2, 'sum': 0}]}]},
  {'group2': 1,
   'groups': [{'group1': 0,
     'groups': [{'group0': 2}, {'group0': 2, 'sum': 3355.5882352941176}]}]},
  {'group2': 0,
   'groups': [{'group1': 0,
     'groups': [{'group0': 1}, {'group0': 1, 'sum': 0}]},
    {'group1': 1,
     'groups': [{'group0': 2}, {'group0': 2, 'sum': 1145.7254901960782}]}]},
  {'group2': 0,
   'groups': [{'group1': 0,
     'groups': [{'group0': 1}, {'group0': 1, 'sum': 0}]},
    {'group1': 1,
     'groups': [{'group0': 1}, {'group0': 1, 'sum': 2218.4941176470584}]}]},
  {'group2': 1,
   'groups': [{'group1': 0,
     'groups': [{'group0': 2}, {'group0': 2, 'sum': 3355.5882352941176}]},
    {'group1': 1,
     'groups': [{'group0': 2}, {'group0': 2, 'sum': 9194.384313725488}]}